# Chapter 4 Exercise 2: TARNet - 深度因果推断的开山之作

## 学习目标

1. 理解 TARNet 的架构设计
2. 实现简化版 TARNet
3. 理解 Factual Loss 的含义
4. 训练和评估 TARNet

---

## TARNet: Treatment-Agnostic Representation Network

### 核心思想

TARNet 是 2017 年提出的开创性工作，首次将深度学习系统地应用于因果效应估计。

**核心设计**：
1. **共享表示层**：学习对处理组和控制组都有用的特征表示
2. **双头输出**：分别预测 Y(0) 和 Y(1)

$$X \xrightarrow{\Phi} \text{Representation} \xrightarrow{\begin{cases} h_0 \to \hat{Y}(0) \\ h_1 \to \hat{Y}(1) \end{cases}}$$

## 生活化类比：双语翻译官

想象你是一个翻译官，需要翻译中文文章给两类读者：
- **英语读者** (处理组)
- **法语读者** (控制组)

**传统方法 (T-Learner)**：
- 雇佣两个独立的翻译官
- 每个翻译官只懂一种语言
- 效率低，且无法利用共同知识

**TARNet 方法**：
- 一个翻译官先理解文章核心含义 (共享表示)
- 然后分别翻译成英语和法语 (双头输出)
- 效率高，且能利用语言间的共同结构

### TARNet 架构图

```
        ┌─────────────────────────────────────────┐
        │         共享表示层 (Shared Repr)          │
        │  X → [Hidden] → [Hidden] → Φ(X)         │
        └─────────────────┬───────────────────────┘
                          │
            ┌─────────────┴─────────────┐
            ▼                           ▼
    ┌───────────────┐           ┌───────────────┐
    │    Head 0     │           │    Head 1     │
    │ Φ(X) → Ŷ(0)  │           │ Φ(X) → Ŷ(1)  │
    └───────────────┘           └───────────────┘
            │                           │
            ▼                           ▼
      控制组预测                    处理组预测
```

In [ ]:
# 导入必要的库
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from typing import Tuple
import matplotlib.pyplot as plt

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

print(f"PyTorch 版本: {torch.__version__}")
print("环境准备完成！")

---

## Part 1: TARNet 架构

### 网络组件

1. **Representation Network (共享)**
   - Input: X (原始特征)
   - Output: Φ(X) (表示)
   - 结构: 多层全连接 + ReLU

2. **Head 0 (控制组)**
   - Input: Φ(X)
   - Output: Ŷ(0)
   - 用于预测不接受处理时的结果

3. **Head 1 (处理组)**
   - Input: Φ(X)
   - Output: Ŷ(1)
   - 用于预测接受处理时的结果

In [ ]:
# 练习 2.1: 理解 TARNet 架构

class SimpleTARNet(nn.Module):
    """
    简化版 TARNet
    
    架构:
    X -> [Shared Representation] -> Phi(X)
                                      |
                    +----------------+----------------+
                    |                                 |
                [Head 0]                         [Head 1]
                    |                                 |
                  Y(0)                              Y(1)
    
    TODO: 完成网络定义
    """
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 50,
        repr_dim: int = 25
    ):
        super().__init__()
        
        # TODO: 定义共享表示层
        # Input -> Hidden (ReLU) -> Representation (ReLU)
        self.representation = nn.Sequential(
            # 你的代码:
            # nn.Linear(input_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, repr_dim),
            # nn.ReLU(),
        )
        
        # TODO: 定义控制组输出头 (Y0)
        # Representation -> Hidden (ReLU) -> Output (1维)
        self.head0 = nn.Sequential(
            # 你的代码:
            # nn.Linear(repr_dim, hidden_dim // 2),
            # nn.ReLU(),
            # nn.Linear(hidden_dim // 2, 1),
        )
        
        # TODO: 定义处理组输出头 (Y1)
        # Representation -> Hidden (ReLU) -> Output (1维)
        self.head1 = nn.Sequential(
            # 你的代码:
            # nn.Linear(repr_dim, hidden_dim // 2),
            # nn.ReLU(),
            # nn.Linear(hidden_dim // 2, 1),
        )
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        前向传播
        
        TODO: 完成前向传播逻辑
        
        Returns:
            (y0_pred, y1_pred, representation)
        """
        # TODO: 计算共享表示
        phi = None  # 你的代码: self.representation(x)
        
        # TODO: 通过两个头计算 Y(0) 和 Y(1)
        y0 = None  # 你的代码: self.head0(phi)
        y1 = None  # 你的代码: self.head1(phi)
        
        return y0, y1, phi
    
    def predict_ite(self, x: torch.Tensor) -> torch.Tensor:
        """
        预测个体处理效应 ITE = Y(1) - Y(0)
        
        TODO: 完成 ITE 预测
        """
        y0, y1, _ = self.forward(x)
        # 你的代码: return y1 - y0
        pass

# 测试架构
model = SimpleTARNet(input_dim=5)
X_sample = torch.randn(10, 5)
try:
    y0, y1, phi = model(X_sample)
    if y0 is not None:
        print(f"模型输出形状: Y0={y0.shape}, Y1={y1.shape}, Phi={phi.shape}")
    else:
        print("[未完成] 请完成 SimpleTARNet.forward 函数")
except Exception as e:
    print(f"[未完成] 请完成 SimpleTARNet 定义: {e}")

---

## Part 2: Factual Loss

### 核心挑战：反事实无法观测

对于每个样本，我们只能观测到一个结果：
- 如果 T=1，我们只观测到 Y(1)
- 如果 T=0，我们只观测到 Y(0)

所以我们只能在 **观测到的结果** 上计算损失！

### Factual Loss 公式

$$L_{factual} = \frac{1}{n}\sum_{i=1}^{n} (Y_i - \hat{Y}_i^{factual})^2$$

其中：
$$\hat{Y}_i^{factual} = \begin{cases} \hat{Y}_i(1) & \text{if } T_i = 1 \\ \hat{Y}_i(0) & \text{if } T_i = 0 \end{cases}$$

In [ ]:
# 练习 2.2: Factual Loss

def compute_factual_loss(
    y_true: torch.Tensor,
    t_true: torch.Tensor,
    y0_pred: torch.Tensor,
    y1_pred: torch.Tensor
) -> torch.Tensor:
    """
    计算 Factual Loss
    
    关键思想: 只在观测到的结果上计算损失
    - 如果 T=1, 损失 = (Y - Y1_pred)^2
    - 如果 T=0, 损失 = (Y - Y0_pred)^2
    
    TODO: 实现 Factual Loss
    
    伪代码:
        y_pred = where(T == 1, Y1_pred, Y0_pred)
        loss = MSE(y_true, y_pred)
    
    Returns:
        scalar loss
    """
    # TODO: 根据处理状态选择预测值
    # 提示: 使用 torch.where(condition, if_true, if_false)
    # t_true 需要 reshape 成和 y0_pred 相同的形状
    
    t_expanded = t_true.unsqueeze(1) if len(t_true.shape) == 1 else t_true
    y_pred = None  # 你的代码: torch.where(t_expanded == 1, y1_pred, y0_pred)
    
    if y_pred is None:
        return None
    
    # TODO: 计算 MSE
    y_true_expanded = y_true.unsqueeze(1) if len(y_true.shape) == 1 else y_true
    loss = None  # 你的代码: torch.mean((y_true_expanded - y_pred)**2)
    
    return loss

# 测试
y_true = torch.FloatTensor([1.0, 2.0, 3.0])
t_true = torch.FloatTensor([1.0, 0.0, 1.0])
y0_pred = torch.FloatTensor([[1.5], [2.0], [2.5]])
y1_pred = torch.FloatTensor([[1.0], [2.5], [3.0]])

loss = compute_factual_loss(y_true, t_true, y0_pred, y1_pred)
if loss is not None:
    print(f"Factual Loss: {loss.item():.4f}")
    print("\n解释:")
    print("样本1: T=1, Y=1.0, Y1_pred=1.0, 误差=0")
    print("样本2: T=0, Y=2.0, Y0_pred=2.0, 误差=0")
    print("样本3: T=1, Y=3.0, Y1_pred=3.0, 误差=0")
else:
    print("[未完成] 请完成 compute_factual_loss 函数")

---

## Part 3: 数据生成

让我们生成一个包含真实潜在结果的数据集，以便评估模型性能。

In [ ]:
# 练习 2.5: 数据生成

def generate_simple_data(
    n: int = 1000,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    生成简单的半合成数据
    
    DGP:
    - X ~ N(0, I) (5维)
    - T ~ Bernoulli(sigmoid(0.5*X1 + 0.3*X2))
    - Y(0) = 1 + 0.5*X1 + 0.3*X2 + noise
    - Y(1) = Y(0) + (2 + 0.5*X1)  # 异质性效应
    
    TODO: 完成数据生成
    
    Returns:
        (X, T, Y, Y0, Y1)
    """
    np.random.seed(seed)
    
    # TODO: 生成特征 (5 维)
    X = None  # 你的代码: np.random.randn(n, 5)
    
    if X is None:
        return None, None, None, None, None
    
    # TODO: 生成处理
    # propensity = sigmoid(0.5*X[:, 0] + 0.3*X[:, 1])
    propensity = None  # 你的代码
    T = None  # 你的代码: np.random.binomial(1, propensity, n)
    
    # TODO: 生成潜在结果
    noise = np.random.randn(n) * 0.5
    # Y(0) = 1 + 0.5*X1 + 0.3*X2 + noise
    Y0 = None  # 你的代码
    # Y(1) = Y(0) + (2 + 0.5*X1)  异质性效应: 基础效应 2, 加上与 X1 相关的部分
    Y1 = None  # 你的代码
    
    # TODO: 观测结果
    Y = None  # 你的代码: np.where(T == 1, Y1, Y0)
    
    return X, T, Y, Y0, Y1

# 测试
X, T, Y, Y0, Y1 = generate_simple_data()
if X is not None and X[0, 0] is not None:
    print(f"数据形状: X={X.shape}")
    print(f"处理比例: {T.mean():.2%}")
    print(f"真实 ATE: {np.mean(Y1 - Y0):.4f}")
    print(f"ITE 范围: [{(Y1-Y0).min():.2f}, {(Y1-Y0).max():.2f}]")
else:
    print("[未完成] 请完成 generate_simple_data 函数")

---

## Part 4: 训练 TARNet

In [ ]:
# 练习 2.3: 训练 TARNet

def train_tarnet(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_epochs: int = 100,
    batch_size: int = 64,
    learning_rate: float = 1e-3
) -> Tuple[SimpleTARNet, dict]:
    """
    训练 TARNet
    
    TODO: 完成训练过程
    
    Returns:
        (trained_model, training_history)
    """
    
    # TODO: 转换为 PyTorch Tensor
    X_tensor = torch.FloatTensor(X)
    T_tensor = torch.FloatTensor(T)
    Y_tensor = torch.FloatTensor(Y)
    
    # TODO: 创建 DataLoader
    dataset = TensorDataset(X_tensor, T_tensor, Y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # TODO: 初始化模型
    model = SimpleTARNet(input_dim=X.shape[1])
    
    # TODO: 定义优化器
    optimizer = None  # 你的代码: optim.Adam(model.parameters(), lr=learning_rate)
    
    if optimizer is None:
        return model, {'loss': []}
    
    # 训练历史
    history = {'loss': []}
    
    # TODO: 训练循环
    for epoch in range(n_epochs):
        epoch_loss = 0
        n_batches = 0
        
        for batch_x, batch_t, batch_y in dataloader:
            # 你的代码:
            # 1. 清零梯度
            # optimizer.zero_grad()
            
            # 2. 前向传播
            # y0_pred, y1_pred, _ = model(batch_x)
            
            # 3. 计算 Factual Loss
            # loss = compute_factual_loss(batch_y, batch_t, y0_pred, y1_pred)
            
            # 4. 反向传播
            # loss.backward()
            
            # 5. 更新参数
            # optimizer.step()
            
            # epoch_loss += loss.item()
            # n_batches += 1
            pass
        
        # 记录损失
        history['loss'].append(epoch_loss / n_batches if n_batches > 0 else 0)
        
        if (epoch + 1) % 20 == 0:
            print(f"  Epoch {epoch+1}/{n_epochs}, Loss: {history['loss'][-1]:.4f}")
    
    return model, history

# 训练
if X is not None:
    print("开始训练 TARNet...")
    model, history = train_tarnet(X, T, Y, n_epochs=100, batch_size=64)
    
    if len(history['loss']) > 0 and history['loss'][-1] > 0:
        print(f"\n训练完成! 最终损失: {history['loss'][-1]:.4f}")
        
        # 绘制训练曲线
        plt.figure(figsize=(8, 4))
        plt.plot(history['loss'])
        plt.xlabel('Epoch')
        plt.ylabel('Factual Loss')
        plt.title('TARNet Training Curve')
        plt.grid(True, alpha=0.3)
        plt.show()
    else:
        print("[未完成] 请完成 train_tarnet 函数")

---

## Part 5: 评估 TARNet

In [ ]:
# 练习 2.4: 评估 TARNet

def evaluate_tarnet(
    model: SimpleTARNet,
    X: np.ndarray,
    Y0_true: np.ndarray,
    Y1_true: np.ndarray
) -> dict:
    """
    评估 TARNet 性能
    
    指标:
    1. PEHE (ITE 误差): sqrt(E[(ITE_true - ITE_pred)^2])
    2. ATE 误差: |ATE_true - ATE_pred|
    
    TODO: 完成评估逻辑
    
    Returns:
        dict with evaluation metrics
    """
    
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        
        # TODO: 预测 Y(0) 和 Y(1)
        y0_pred, y1_pred, _ = model(X_tensor)
        
        if y0_pred is None:
            return {'pehe': None, 'ate_true': None, 'ate_pred': None, 'ate_error': None}
        
        y0_pred = y0_pred.squeeze().numpy()
        y1_pred = y1_pred.squeeze().numpy()
    
    # TODO: 计算 PEHE
    # PEHE = sqrt(mean((ITE_true - ITE_pred)^2))
    ite_true = Y1_true - Y0_true
    ite_pred = y1_pred - y0_pred
    pehe = None  # 你的代码: np.sqrt(np.mean((ite_true - ite_pred)**2))
    
    # TODO: 计算 ATE 误差
    ate_true = np.mean(Y1_true - Y0_true)
    ate_pred = np.mean(y1_pred - y0_pred)
    ate_error = None  # 你的代码: np.abs(ate_true - ate_pred)
    
    return {
        'pehe': pehe,
        'ate_true': ate_true,
        'ate_pred': ate_pred,
        'ate_error': ate_error
    }

# 评估
if X is not None and model is not None:
    metrics = evaluate_tarnet(model, X, Y0, Y1)
    if metrics['pehe'] is not None:
        print("=" * 40)
        print("TARNet 评估结果")
        print("=" * 40)
        print(f"PEHE: {metrics['pehe']:.4f}")
        print(f"真实 ATE: {metrics['ate_true']:.4f}")
        print(f"预测 ATE: {metrics['ate_pred']:.4f}")
        print(f"ATE 误差: {metrics['ate_error']:.4f}")
    else:
        print("[未完成] 请完成 evaluate_tarnet 函数")

---

## Part 6: 可视化结果

In [ ]:
# 可视化 ITE 预测

if X is not None and model is not None:
    model.eval()
    with torch.no_grad():
        y0_pred, y1_pred, _ = model(torch.FloatTensor(X))
        if y0_pred is not None:
            ite_pred = (y1_pred - y0_pred).squeeze().numpy()
            ite_true = Y1 - Y0
            
            fig, axes = plt.subplots(1, 3, figsize=(15, 4))
            
            # 图1: ITE 真实 vs 预测
            axes[0].scatter(ite_true, ite_pred, alpha=0.5, s=10)
            axes[0].plot([ite_true.min(), ite_true.max()], 
                        [ite_true.min(), ite_true.max()], 'r--', lw=2)
            axes[0].set_xlabel('True ITE')
            axes[0].set_ylabel('Predicted ITE')
            axes[0].set_title('ITE: True vs Predicted')
            
            # 图2: ITE 分布
            axes[1].hist(ite_true, bins=30, alpha=0.5, label='True', density=True)
            axes[1].hist(ite_pred, bins=30, alpha=0.5, label='Predicted', density=True)
            axes[1].set_xlabel('ITE')
            axes[1].set_ylabel('Density')
            axes[1].set_title('ITE Distribution')
            axes[1].legend()
            
            # 图3: ITE 与特征的关系
            axes[2].scatter(X[:, 0], ite_true, alpha=0.5, s=10, label='True')
            axes[2].scatter(X[:, 0], ite_pred, alpha=0.5, s=10, label='Predicted')
            axes[2].set_xlabel('X1 (第一个特征)')
            axes[2].set_ylabel('ITE')
            axes[2].set_title('ITE vs X1')
            axes[2].legend()
            
            plt.tight_layout()
            plt.show()
        else:
            print("请先完成模型训练")

---

## 思考题

### 1. 为什么 TARNet 需要两个独立的输出头?

**你的答案:**


### 2. Factual Loss 和普通的监督学习损失有什么区别?

**你的答案:**


### 3. 如果我们只训练一个头 (比如只训练 Head 1)，会有什么问题?

**你的答案:**


### 4. 共享表示层的作用是什么? 为什么不给两个头分别的特征提取器?

**你的答案:**


### 5. TARNet 如何预测反事实结果 (比如对于 T=1 的人预测 Y(0))?

**你的答案:**


### 6. 在什么情况下 TARNet 会比传统的 Meta-Learner (如 S/T-Learner) 更好?

**你的答案:**


---

## 总结

| 概念 | 要点 |
|------|------|
| **TARNet** | 首个系统化的深度因果模型 |
| **共享表示** | 让两个头共享底层特征提取 |
| **双头输出** | 分别预测 Y(0) 和 Y(1) |
| **Factual Loss** | 只在观测结果上计算损失 |
| **PEHE** | 评估 ITE 预测精度的黄金指标 |

### 下一步

在下一个练习中，我们将学习 **DragonNet** —— TARNet 的增强版，加入了倾向得分预测！

---

*恭喜你完成了 TARNet 的学习！* 🎉